<a href="https://colab.research.google.com/github/Rocking-Priya/Mod-C-Summer-2025-Week-1--6-assignment/blob/main/Mod_C_Week_1_%E2%80%93_Linear_Regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK FOR WEEK 1 -  Include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features.

In [3]:
import pandas as pd

In [4]:
phishing_df_1 = pd.read_csv('/content/dataset_phishing.csv')
phishing_df_1.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [5]:
phishing_df_1.isnull().sum()

,0
url,0
length_url,0
length_hostname,0
ip,0
nb_dots,0
...,...
web_traffic,0
dns_record,0
google_index,0
page_rank,0


In [7]:
X = phishing_df_1.drop('status', axis=1)
y = phishing_df_1['status']

In [8]:
# Check data types of columns in X
print("Data types of X before VIF calculation:")
print(X.dtypes)

Data types of X before VIF calculation:
url                object
length_url          int64
length_hostname     int64
ip                  int64
nb_dots             int64
                    ...  
domain_age          int64
web_traffic         int64
dns_record          int64
google_index        int64
page_rank           int64
Length: 88, dtype: object


In [6]:
import numpy as np

In [9]:
X_numeric = X.select_dtypes(include=np.number)

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [11]:
X_with_const = add_constant(X_numeric) # Use the numeric DataFrame. This adds a column of 1s (called "const") to your data so the model includes an intercept term.

vif = pd.DataFrame()
vif["Variable"] = X_with_const.columns #This part calculates the VIF for each variable (column).

# Calculate VIF, ensuring we use the numeric values
vif["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])] #VIF shows how much a variable is related to other variables.
vif

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,Variable,VIF
0,const,124.747685
1,length_url,11.469510
2,length_hostname,3.941609
3,ip,3.413926
4,nb_dots,8.152036
...,...,...
83,domain_age,1.875603
84,web_traffic,1.244188
85,dns_record,1.652678
86,google_index,1.967224


VIF > 10 means strong multicollinearity. That variable might be too similar to others and could mess up your model.

Example: length_url has a VIF of 11.47, so it might be giving similar information as other variables. You might consider removing or combining it with related features.

High VIF means the model might not be trustworthy.

Multicollinearity can make it hard to tell which variable is really important.



In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Convert the target variable 'y' to numerical values
# Map 'phishing' to 1 and 'legitimate' to 0 (or vice versa)
y_encoded = y.map({'phishing': 1, 'legitimate': 0})

# Select only numeric columns for X
# Keep this as before, ensuring only numeric features are used for the model
X_numeric = X.select_dtypes(include=np.number)

# Split data using the numeric DataFrame for X and the encoded target variable y
X_train, X_test, y_train, y_test = train_test_split(X_numeric, y_encoded, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("R² score:", r2_score(y_test, y_pred))

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate RMSE by taking the square root of the MSE
print("RMSE:", np.sqrt(mse))

R² score: 0.7442130858002536
RMSE: 0.25285793430805237


You've trained a linear regression model to predict whether a website is phishing or legitimate, based on the numeric features of the website. You've evaluated its performance using two metrics: R-squared (R²) and Root Mean Squared Error (RMSE).

Here's what those values mean:

R² score: 0.7442130858002536

The R-squared value, also known as the coefficient of determination, represents the proportion of the variance in the dependent variable (whether the website is phishing or legitimate) that is predictable from the independent variables (the numeric features).
An R² of 0.744 (or approximately 74.4%) means that your model explains about 74.4% of the variability in the target variable.
In simpler terms, your model does a reasonably good job of capturing the relationship between the website features and its status (phishing or legitimate). A higher R² generally indicates a better fit of the model to the data, though a high R² doesn't necessarily mean the model is perfect or that the variables are the only factors.
RMSE: 0.25285793430805237

RMSE stands for Root Mean Squared Error. It measures the average magnitude of the errors between the predicted values and the actual values.
The unit of RMSE is the same as the unit of the dependent variable. In your case, your dependent variable y_encoded is either 0 (legitimate) or 1 (phishing).
An RMSE of approximately 0.253 means that, on average, your model's predictions are about 0.253 units away from the actual value (0 or 1).
Since your target variable is binary (0 or 1), an RMSE of 0.253 indicates that the model's predictions, which are continuous values between 0 and 1, deviate from the true binary status by this amount on average. A lower RMSE indicates that the model's predictions are closer to the actual values, which is desirable.
In summary:

Your model is able to explain a significant portion (about 74.4%) of the variation in whether a website is phishing or not based on its numeric features.
The average error in your model's predictions, in terms of the 0/1 scale, is about 0.253.
These metrics suggest that your linear regression model has some predictive power for classifying websites as phishing or legitimate based on the numeric features, although it's not a perfect prediction. Since your target variable is binary, it's worth noting that while you've used a linear regression model (which is typically for continuous outcomes), the results are interpreted in the context of predicting a value between 0 and 1. For binary classification problems, metrics like accuracy, precision, recall, F1-score, and ROC AUC are often more appropriate, and models designed for classification (like Logistic Regression, Support Vector Machines, or Tree-based models) are typically used.

In [15]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

# Select only numeric columns from X before applying PolynomialFeatures
X_numeric = X.select_dtypes(include=np.number)
X_poly = poly.fit_transform(X_numeric)

# Convert the target variable 'y' to numerical values
# Map 'phishing' to 1 and 'legitimate' to 0 (or vice versa)
y_encoded = y.map({'phishing': 1, 'legitimate': 0})

# Train model again, using the polynomial features and encoded target variable
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y_encoded, test_size=0.2, random_state=42)
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Predict and evaluate
y_pred_poly = model_poly.predict(X_test_poly)
print("R² (Poly):", r2_score(y_test, y_pred_poly))
# Calculate MSE
mse_poly = mean_squared_error(y_test, y_pred_poly)
# Calculate RMSE by taking the square root of the MSE
print("RMSE (Poly):", np.sqrt(mse_poly))

R² (Poly): 0.7085155193388634
RMSE (Poly): 0.2699262634055965
